## self made code blocks
copy the codes from the websites, and try to get rid of useless part.

In [5]:
import pandas as pd
import tiktoken
from utils.embeddings_utils import get_embedding

ModuleNotFoundError: No module named 'utils'

In [6]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191


In [8]:
# load & inspect dataset
# input_datapath = "data/fine_food_reviews_1k.csv"  # to save space, we provide a pre-filtered dataset
input_datapath = "/Users/timmy/Desktop/LLM_prompts/review_data/Reviews.csv"
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(2)


,Time,ProductId,UserId,Score,Summary,Text,combined
Id,,,,,,,
1,1303862400,B001E4KFG0,A3SGXH7AUHU8GW,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Title: Good Quality Dog Food; Content: I have ...
2,1346976000,B00813GRG4,A1D87F6ZCVE5NK,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Title: Not as Advertised; Content: Product arr...


In [9]:
# subsample to 1k most recent reviews and remove samples that are too long
top_n = 1000
df = df.sort_values("Time").tail(top_n * 2)  # first cut to first 2k entries, assuming less than half will be filtered out
df.drop("Time", axis=1, inplace=True)

encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
len(df)


1000

In [24]:
encoding

<Encoding 'cl100k_base'>

In [16]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage

# This may take a few minutes
# df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, model=embedding_model))
df.to_csv("fine_food_reviews_with_embeddings_1k.csv")


In [4]:
output_datapath = "fine_food_reviews_with_embeddings_1k.csv"
df_output = pd.read_csv(output_datapath, index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: 'fine_food_reviews_with_embeddings_1k.csv'

In [25]:
df_output[:2]

,ProductId,UserId,Score,Summary,Text,combined,n_tokens,embedding
Id,,,,,,,,
284932,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,52,"[0.007060592994093895, -0.02732112631201744, 0..."
220697,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...,35,"[-0.023609420284628868, -0.011784634552896023,..."


In [1]:
len(df_output.iloc[0]['embedding'])

NameError: name 'df_output' is not defined

In [27]:
len(df_output.iloc[1]['embedding'])

34399